In [6]:
from datetime import datetime,timedelta

In [16]:
exec_datetime = '2019-12-07 07:21:32'
wrf_run = '0'
wrf_run = int(wrf_run)
gfs_hour = '06'
exec_datetime = datetime.strptime(exec_datetime, '%Y-%m-%d %H:%M:%S')
print(exec_datetime)
exec_date_str = exec_datetime.strftime('%Y-%m-%d')
exec_date = datetime.strptime(exec_date_str, '%Y-%m-%d')
print(exec_date)

2019-12-07 07:21:32
2019-12-07 00:00:00


In [17]:
ts_start_date = exec_date - timedelta(days=wrf_run)
ts_start_date_str = ts_start_date.strftime('%Y-%m-%d')
print(ts_start_date_str)

2019-12-07


In [19]:
gfs_ts_start_utc_str = '{} {}:00:00'.format(ts_start_date_str, gfs_hour)
print(gfs_ts_start_utc_str)

2019-12-07 06:00:00


In [20]:
gfs_ts_start_utc = datetime.strptime(gfs_ts_start_utc_str, '%Y-%m-%d %H:%M:%S')
gfs_ts_start_local = gfs_ts_start_utc + timedelta(hours=5, minutes=30)
print(gfs_ts_start_local)

2019-12-07 11:30:00


In [21]:
gfs_ts_end_local = gfs_ts_start_local + timedelta(days=3)
print(gfs_ts_end_local)

2019-12-10 11:30:00
